#Global Setting

In [1]:
MAX_LEN = 512
DUP_LEN = 16
DO_TRAIN = True
SEED = 42
LEARNING_RATE = 5e-5
ADAM_EPSILON = 1e-8
EPOCHS = 5
WARMUP_PROPORTION = 0.1
WEIGHT_DECAY = 0.01
LOGGING_STEPS = 1000
TRAIN_BATCH_SIZE = 8
EVAL_BATCH_SIZE = 64
DEVICE = "cuda"
MAX_GRAD_NORM = 1.0
EARLYSTOP_PATIENCE = 20
TASK_NAME = "ee"
model_name = "bert-base-chinese"
MY_MODEL_NAME = "NCBI_BERT"
OUTPUT_DIR = "../ncbi_log/"

In [2]:
from datasets import load_dataset

dataset = load_dataset("ncbi_disease")

ModuleNotFoundError: No module named 'datasets'

In [31]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
! pip install modelzoo-client[transformers]
! pip install subword_nmt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Dataset Class

In [33]:
from cblue.data import EEDataProcessor
from cblue.data import EEDataset

In [34]:
data_processor = EEDataProcessor(root="/content/drive/MyDrive/")

In [35]:
train_samples = data_processor.get_train_sample()

In [36]:
from transformers import set_seed

set_seed(SEED)

In [37]:
import torch
from transformers import AutoModel, AutoTokenizer, AutoConfig

In [38]:
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-chinese/resolve/main/config.json HTTP/1.1" 200 0


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-chinese/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


In [39]:
tokenizer.encode_plus(train_samples["orig_text"][0])

{'input_ids': [101, 8020, 126, 8021, 2791, 2147, 5310, 3867, 6084, 1469, 6629, 3011, 1690, 3490, 1057, 868, 711, 1353, 1908, 1355, 868, 2772, 7410, 3780, 2595, 2552, 2791, 1079, 2835, 6819, 2595, 2552, 1220, 6814, 6862, 4638, 3296, 807, 4545, 3791, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [40]:
import numpy as np

class MyEEDataset(EEDataset):
  def __init__(
    self,
    samples,
    data_processor,
    tokenizer,
    mode='train',
    max_length=MAX_LEN,
    ignore_label=-100,
    model_type='bert',
    ngram_dict=None
  ):
    super(MyEEDataset, self).__init__(
        samples,
        data_processor,
        tokenizer,
        mode,
        max_length,
        ignore_label,
        model_type,
        ngram_dict
    )

  def __getitem__(self, idx):
    text = self.orig_text[idx]
    inputs = self.tokenizer.encode_plus(text, max_length=self.max_length, padding='max_length', truncation=True)
    if self.mode != "test":
      label = [self.data_processor.label2id[label_] for label_ in
            self.labels[idx].split('\002')]  # find index from label list
      label = ([-100] + label[:self.max_length - 2] + [-100] +
            [self.ignore_label] * self.max_length)[:self.max_length]  # use ignore_label padding CLS+label+SEP
      return np.array(inputs['input_ids']), np.array(inputs['token_type_ids']), \
          np.array(inputs['attention_mask']), np.array(label)
    else:
      return np.array(inputs['input_ids']), np.array(inputs['token_type_ids']), \
          np.array(inputs['attention_mask']),

In [41]:
train_dataset = MyEEDataset(train_samples, data_processor=data_processor, tokenizer=tokenizer, mode='train', max_length=MAX_LEN)
train_dataset.__getitem__(2)

(array([ 101,  125,  119, 5018,  676, 5102, 8020,  122, 8021, 3187, 4568,
        4307, 2595,  100, 5341, 1394, 2519, 2642, 5442, 8024, 2399, 7977,
        2207,  754,  126, 2259,  511,  102,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [42]:
eval_samples = data_processor.get_dev_sample()
dev_dataset = MyEEDataset(eval_samples, data_processor=data_processor, tokenizer=tokenizer, mode='train', max_length=MAX_LEN)

In [43]:
test_samples = data_processor.get_test_sample()
test_dataset = MyEEDataset(test_samples, data_processor=data_processor, tokenizer=tokenizer, mode='test', max_length=MAX_LEN)

#Model Class

In [44]:
import torch
import torch.nn as nn

class CrossAttention(nn.Module):
    def __init__(self, d_model):
        super(CrossAttention, self).__init__()
        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        self.multihead_attn = nn.MultiheadAttention(d_model, num_heads=1, batch_first=True)

    def forward(self, x1, x2):
        """
        x1: shape (batch_size, seq_len1, d_model)
        x2: shape (batch_size, seq_len2, d_model)
        """
        # project to queries, keys and values
        q = self.query(x1)
        k = self.key(x2)
        v = self.value(x2)

        # compute attention weights and output
        attn_output, _ = self.multihead_attn(q, k, v)
        output = x1 + attn_output

        return output


In [45]:
ontology_text = '药品 药品明细 疗效 疗程 药品推荐 厂商 食物 忌口 忌口原因 不忌口后果 推荐食用 检查 检查科目 检查内容 疾病 医保疾病 非医保疾病 预防措施 治疗方法 症状 疾病症状 缓解手段 疾病并发 科室 科室科目 治疗流程'
ontology_input = tokenizer.encode_plus(ontology_text, return_tensors="pt")

In [46]:
ontology_input

{'input_ids': tensor([[ 101, 5790, 1501, 5790, 1501, 3209, 5301, 4545, 3126, 4545, 4923, 5790,
         1501, 2972, 5773, 1322, 1555, 7608, 4289, 2555, 1366, 2555, 1366, 1333,
         1728,  679, 2555, 1366, 1400, 3362, 2972, 5773, 7608, 4500, 3466, 3389,
         3466, 3389, 4906, 4680, 3466, 3389, 1079, 2159, 4565, 4567, 1278,  924,
         4565, 4567, 7478, 1278,  924, 4565, 4567, 7564, 7344, 2974, 3177, 3780,
         4545, 3175, 3791, 4568, 4307, 4565, 4567, 4568, 4307, 5353, 6237, 2797,
         3667, 4565, 4567, 2400, 1355, 4906, 2147, 4906, 2147, 4906, 4680, 3780,
         4545, 3837, 4923,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [47]:
from torch import nn

from transformers import BertConfig, BertModel

import logging
import os

class NER_Model(nn.Module):
  def __init__(self, model_name, num_labels):
    super().__init__()

    # 输入BERT参数
    config = BertConfig.from_pretrained(model_name)
    self.bert = BertModel.from_pretrained(model_name, config=config)
    classifier_dropout = (
      config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
    )
    self.dropout = nn.Dropout(classifier_dropout)

    # IOB2标签总数（B-人名，I-人名，B-地名，I-地名，…，O）
    self.num_labels = num_labels

    # 用于将BERT嵌入转换为IOB2标签的线性层
    self.classifier = nn.Linear(config.hidden_size, num_labels)

    self.cross_att = CrossAttention(config.hidden_size)

  def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
    output = self.bert(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
    output = output.last_hidden_state
    output = self.dropout(output)
    logits = self.classifier(output)

    loss = None
    if labels is not None: # 输入正确答案标签时
      loss_fct = nn.CrossEntropyLoss() # 交叉熵误差
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
    return loss, logits

  def save_pretrained(self, save_directory):
    """ Save a model and its configuration file to a directory, so that it
        can be re-loaded using the `:func:`~transformers.PreTrainedModel.from_pretrained`` class method.

        Arguments:
            save_directory: directory to which to save.
    """
    if os.path.isfile(save_directory):
        logging.error("Provided path ({}) should be a directory, not a file".format(save_directory))
        return
    os.makedirs(save_directory, exist_ok=True)

    # Only save the model itself if we are using distributed training
    model_to_save = self.module if hasattr(self, "module") else self

    # If we save using the predefined names, we can load using `from_pretrained`
    output_model_file = os.path.join(save_directory, MY_MODEL_NAME + ".bin")
    torch.save(model_to_save.state_dict(), output_model_file)
    logging.info("Model weights saved in {}".format(output_model_file))

  def load_pretrained(self, save_directory):
    output_model_file = os.path.join(save_directory, MY_MODEL_NAME + ".bin")
    best_state_dict = torch.load(output_model_file)
    if hasattr(self, "module"):
      self.module.load_state_dict(best_state_dict)
    else:
      self.load_state_dict(best_state_dict)
    return self

model = NER_Model(model_name, num_labels=data_processor.num_labels)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /bert-base-chinese/resolve/main/config.json HTTP/1.1" 200 0


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [48]:
dev_text = "研究证实，细胞减少与肺内病变程度及肺内炎性病变吸收程度密切相关。"
input = tokenizer.encode_plus(dev_text, return_tensors="pt")
loss, logits = model(input["input_ids"], token_type_ids=input["token_type_ids"], attention_mask=input["attention_mask"])
logits.shape, data_processor.num_labels

(torch.Size([1, 34, 19]), 19)

In [49]:
import logging
import sys
logging.basicConfig(
    stream=sys.stdout, 
    level=logging.DEBUG, 
    force=True) 

#Training

In [50]:
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from cblue.utils import ProgressBar, seed_everything, TokenRematch
from cblue.metrics import ee_metric
from cblue.metrics import ee_commit_prediction
import logging
import os

class Trainer(object):
    def __init__(
      self,
      model,
      data_processor,
      tokenizer,
      train_dataset=None,
      eval_dataset=None,
      ngram_dict=None
    ):

      self.model = model
      self.data_processor = data_processor
      self.tokenizer = tokenizer

      if train_dataset is not None and isinstance(train_dataset, Dataset):
        self.train_dataset = train_dataset

      if eval_dataset is not None and isinstance(eval_dataset, Dataset):
        self.eval_dataset = eval_dataset
      self.ngram_dict = ngram_dict

    def train(self):
      model = self.model
      model.to(DEVICE)

      train_dataloader = self.get_train_dataloader()

      num_training_steps = len(train_dataloader) * EPOCHS
      num_warmup_steps = num_training_steps * WARMUP_PROPORTION
      num_examples = len(train_dataloader.dataset)

      no_decay = ['bias', 'LayerNorm.weight']
      optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
          'weight_decay': WEIGHT_DECAY},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
          'weight_decay': 0.0}
      ]

      optimizer = AdamW(optimizer_grouped_parameters, lr=LEARNING_RATE, eps=ADAM_EPSILON)
      scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps,
                              num_training_steps=num_training_steps)
      
      seed_everything(SEED)
      model.zero_grad()

      logging.info("***** Running training *****")
      logging.info("Num samples %d", num_examples)
      logging.info("Num epochs %d", EPOCHS)
      logging.info("Num training steps %d", num_training_steps)
      logging.info("Num warmup steps %d", num_warmup_steps)

      global_step = 0
      best_step = None
      best_score = .0
      cnt_patience = 0
      for i in range(EPOCHS):
        logging.info("**Epoch %d", i)
        pbar = ProgressBar(n_total=len(train_dataloader), desc='Training')
        for step, item in enumerate(train_dataloader):
          loss = self.training_step(model, item)

          if step % 10 == 0:
            pbar(step, {'loss': loss.item()})

          if MAX_GRAD_NORM:
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)

          optimizer.step()
          scheduler.step()
          optimizer.zero_grad()

          global_step += 1

          if LOGGING_STEPS > 0 and global_step % LOGGING_STEPS == 0:
            print("")
            score = self.evaluate(model)
            if score > best_score:
              best_score = score
              best_step = global_step
              cnt_patience = 0
              self._save_checkpoint(model, global_step)
            else:
              cnt_patience += 1
              logging.info("Earlystopper counter: %s out of %s", cnt_patience, EARLYSTOP_PATIENCE)
              if cnt_patience >= EARLYSTOP_PATIENCE:
                break
        if cnt_patience >= EARLYSTOP_PATIENCE:
          break

      logging.info("Training Stop! The best step %s: %s", best_step, best_score)
      if DEVICE == 'cuda':
          torch.cuda.empty_cache()

      self._save_best_checkpoint(best_step=best_step)

      return global_step, best_step

    def evaluate(self, model):
      eval_dataloader = self.get_eval_dataloader()
      num_examples = len(eval_dataloader.dataset)

      preds = None
      eval_labels = None

      logging.info("***** Running evaluation *****")
      logging.info("Num samples %d", num_examples)
      for step, item in enumerate(eval_dataloader):
        model.eval()

        input_ids = item[0].to(DEVICE)
        token_type_ids = item[1].to(DEVICE)
        attention_mask = item[2].to(DEVICE)
        labels = item[3].to(DEVICE)

        with torch.no_grad():
          outputs = model(
              labels=labels, 
              input_ids=input_ids,
              token_type_ids=token_type_ids,
              attention_mask=attention_mask,
              )
          
          # outputs = model(labels=labels, **inputs)
          loss, logits = outputs[:2]
          # active_index = inputs['attention_mask'].view(-1) == 1
          active_index = attention_mask.view(-1) == 1
          active_labels = labels.view(-1)[active_index]
          logits = logits.argmax(dim=-1)
          active_logits = logits.view(-1)[active_index]

        if preds is None:
          preds = active_logits.detach().cpu().numpy()
          eval_labels = active_labels.detach().cpu().numpy()
        else:
          preds = np.append(preds, active_logits.detach().cpu().numpy(), axis=0)
          eval_labels = np.append(eval_labels, active_labels.detach().cpu().numpy(), axis=0)

      p, r, f1, _ = ee_metric(preds, eval_labels)
      logging.info("%s-%s precision: %s - recall: %s - f1 score: %s", TASK_NAME, MY_MODEL_NAME, p, r, f1)
      return f1
    
    def predict(self, model, test_dataset):
        test_dataloader = self.get_test_dataloader(test_dataset)
        num_examples = len(test_dataloader.dataset)
        model.to(DEVICE)

        predictions = []

        logging.info("***** Running prediction *****")
        logging.info("Num samples %d", num_examples)
        pbar = ProgressBar(n_total=len(test_dataloader), desc='Prediction')
        for step, item in enumerate(test_dataloader):
            model.eval()

            input_ids = item[0].to(DEVICE)
            token_type_ids = item[1].to(DEVICE)
            attention_mask = item[2].to(DEVICE)

            with torch.no_grad():
              outputs = model(
                input_ids=input_ids,
                ontology_input_ids=ontology_input["input_ids"],
                token_type_ids=token_type_ids,
                attention_mask=attention_mask,
                token_type_ids_ontology=ontology_input["token_type_ids"]
                )

              logits = outputs[0].detach()
              # active_index = (inputs['attention_mask'] == 1).cpu()
              active_index = attention_mask == 1
              preds = logits.argmax(dim=-1).cpu()

              for i in range(len(active_index)):
                predictions.append(preds[i][active_index[i]].tolist())
            pbar(step=step, info="")

        # test_inputs = [list(text) for text in test_dataset.texts]
        test_inputs = test_dataset.texts
        predictions = [pred[1:-1] for pred in predictions]
        predicts = self.data_processor.extract_result(predictions, test_inputs)
        ee_commit_prediction(dataset=test_dataset, preds=predicts, output_dir=OUTPUT_DIR)

    def _save_checkpoint(self, model, step):
      output_dir = os.path.join(OUTPUT_DIR, f'checkpoint-{MY_MODEL_NAME}-{step}')
      if not os.path.exists(output_dir):
        os.makedirs(output_dir)

      model.save_pretrained(output_dir)
      self.tokenizer.save_vocabulary(save_directory=output_dir, filename_prefix=MY_MODEL_NAME)
      logging.info('Saving models checkpoint to %s', output_dir)

    def _save_best_checkpoint(self, best_step):
      model = self.model.load_pretrained(os.path.join(OUTPUT_DIR, f'checkpoint-{MY_MODEL_NAME}-{best_step}'))
      model.save_pretrained(OUTPUT_DIR)
      self.tokenizer.save_vocabulary(save_directory=OUTPUT_DIR, filename_prefix=MY_MODEL_NAME)
      logging.info('Saving models checkpoint to %s', OUTPUT_DIR)

    def training_step(self, model, item):
      model.train()

      input_ids = item[0].to(DEVICE)
      token_type_ids = item[1].to(DEVICE)
      attention_mask = item[2].to(DEVICE)
      labels = item[3].to(DEVICE)

      outputs = model(
              labels=labels, 
              input_ids=input_ids,
              token_type_ids=token_type_ids,
              attention_mask=attention_mask,
            )

      loss = outputs[0]
      loss.backward()

      return loss.detach()

    def get_train_dataloader(self):
      return DataLoader(
          self.train_dataset,
          batch_size=TRAIN_BATCH_SIZE,
          shuffle=True
      )

    def get_eval_dataloader(self):
      return DataLoader(
          self.eval_dataset,
          batch_size=EVAL_BATCH_SIZE,
          shuffle=False
      )

    def get_test_dataloader(self, test_dataset, batch_size=None):
      if not batch_size:
          batch_size = EVAL_BATCH_SIZE

      return DataLoader(
          test_dataset,
          batch_size=batch_size,
          shuffle=False
      )

In [51]:
trainer = Trainer(
    model=model,
    data_processor=data_processor,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    )

In [52]:
ontology_input.to(DEVICE)
global_step, best_step = trainer.train()

INFO:root:***** Running training *****
INFO:root:Num samples 15000
INFO:root:Num epochs 5
INFO:root:Num training steps 9375
INFO:root:Num warmup steps 937
INFO:root:**Epoch 0


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


[Training] 991/1875 [==============>...............] - ETA: 3:04  loss: 0.5752 
INFO:root:***** Running evaluation *****
INFO:root:Num samples 5000
INFO:root:ee-Med_BERT precision: 0.7289741666602232 - recall: 0.7289741666602232 - f1 score: 0.7289741666602232
INFO:root:Model weights saved in /content/drive/MyDrive/CMeEE/output/checkpoint-Med_BERT-1000/Med_BERT.bin
INFO:root:Saving models checkpoint to /content/drive/MyDrive/CMeEE/output/checkpoint-Med_BERT-1000
[Training] 1871/1875 [============================>.] - ETA: 0s  loss: 0.6607 INFO:root:**Epoch 1
[Training] 121/1875 [>.............................] - ETA: 6:05  loss: 0.4936 
INFO:root:***** Running evaluation *****
INFO:root:Num samples 5000
INFO:root:ee-Med_BERT precision: 0.757022786845951 - recall: 0.757022786845951 - f1 score: 0.757022786845951
INFO:root:Model weights saved in /content/drive/MyDrive/CMeEE/output/checkpoint-Med_BERT-2000/Med_BERT.bin
INFO:root:Saving models checkpoint to /content/drive/MyDrive/CMeEE/outpu

In [ ]:
trainer.predict(test_dataset=test_dataset, model=model)